# Metody Głębokiego uczenia
# Raport - Projekt 1

* *Aleksandra Miesiąc*
* *Agata Pałdyna* 
* *Katarzyna Spalińska*

In [ ]:
@ TODO : 
    
    RAPORT
1) Wstęp (coś tam że ten raport dotyczy projektu zrealizowanego w ramach przedmiotu MGU, 
          na czym polegał projekt, że został napisany pogram, co robi ten program, do czego służy)
2) Opis programu (architektura, klasy, funkcje, plik konfiguracyjny, itp)
        2.1) 
        2.2)
    ...
3) Przeprowadzone eksperymenty (wykresy itp)
4) Wnioski

# Opis architektury programu

Program składa się z kilku modułów oraz pliku konfiguracyjnego, służącego do ustalenia parametrów sieci, opisanego w kolejnym punkcie raportu. 

__Plik *activation_functions.py*__ zawiera implementacje 11 różnych funkcji aktywacji wraz z ich pochodnymi, tak aby można było swobodnie zmieniać ten parametr sieci. Zadana w pliku konfiguracyjnym funkcja aktywacji jest przypisana wszystkim neuronom wewnątrz sieci w przypadku problemu klasyfikacji, zaś wszystkim neuronom warstw ukrytych w przypadku problemu regresji (neurony warstwy wejściowej i wyjściowej mają funkcję aktywacji "ReLU").


__Plik *neural_network.py*__ zawiera implementacje dwóch klas: NeuralNetwork oraz Layer.

__Klasa Layer__
Implementuje obiekt "warstwa", której atrybutami są:
* rozmiar warstwy
* wektor wag (podczas tworzenia generowany losowo)
* wektor wartości neuronów (podczas tworzenia pusty)

__Klasa NeuralNetwork__
Implementuje obiekt "sieć neuronowa", której atrybutami są:
* *input_vector* - wektor wejściowy sieci
* *output_vector* - wektor wyjściowy, obliczony przez sieć
* *output* - oczekiwane wyjście sieci
* *layers* - wektor warstw (obiektów typu Layer)
* *number_of_layers* - liczba warstw (z pliku konfiguracyjnego)
* *layer_size* - liczba neuronów w warstwie (z pliku konfiguracyjnego)
* *activation_function* - funkcja aktywacji (z pliku konfiguracyjnego)
* *learning_rate* - współczynnik uczenia (z pliku konfiguracyjnego)
* *problem* - rodzaj problemu, dla którego sieć będzie trenowana (z pliku konfiguracyjnego)
* *number_of_samples*

Zawiera funkcje:
* *add_layers()* - budującą strukturę sieci. Tworzy odpowiednie obiekty klasy Layer i umieszcza je w wektorze warstw.
* *backpropagation()* - funkcja implementująca algorytm propagacji wstecznej
* *loss_function()* - funkcja obliczająca błąd sieci według wzoru: (norm(y' - y))^2
* *feedforward()* - funkcja implementująca algorytm przechodzenia przez sieć od wejścia do wyjścia
* *argmax_output()* - funkcja która transformuje macierzowe wyjście sieci, zakodowane metodą 'one-hot encoding', na pojedynczy wektor o wartościach takich jak klasy w wektorze oczekiwanego wyjścia sieci (wybierając klasę o największym prawdopodobieństwie)
* *train()* - funkcja trenowania sieci, w której wywołane są raz, kolejno: *feedforward()*, *backpropagation()*
* *predict()* - funkcja implementująca predykcję na nauczonej sieci, wywołująca raz funkcję *feedforward()* i zwracająca wektor wyjściowy sieci (dla problemu regresji) oraz przerobiony funkcją *argmax_output* wektor wyjściowy sieci dla problemu klasyfikacji.
* evaluate()* - funkcja obliczająca procent poprawnych odpowiedzi sieci

__Plik *main.py*__

W pliku main.py następuje kolejno:
* wczytanie pliku konfiguracyjnego
* wczytanie plików ze zbiorami treningowymi i testowymi
* podzielenie zbiorów danych na podzbiory train_X, train_y, test_X, test_y
* podzielenie zbiorów danych na podzbiory (*batch*)
* utworzenie sieci neuronowej zgodnie z plikiem konfiguracyjnym
* iteracyjne trenowanie sieci
* ewaluacja wyników wraz z wygenerowaniem wykresów

# Opis pliku konfiguracyjnego:

Parametry sieci oraz ustawienia programu można modyfikować za pomocą pliku configuration_file.txt, w którym dane zapisane są w postaci nazwa_parametru=wartość. Znajdują się w nim następujące pola:
* *number_of_layers* – liczba ukrytych warstw; liczba naturalna
* *layer_size* – rozmiar warstw (liczba neuronów w warstwie); liczba naturalna lub lista liczb naturalnych oddzielonych przecinkami (np. 3,5,2)
* *activation_function* – nazwa funkcji aktywacji
* *number_of_iterations* – liczba iteracji/epok; liczba naturalna
* *learning_rate* – wartość współczynnika nauki; liczba rzeczywista dodatnia
* *batch_size* – rozmiar batcha; liczba naturalna
* *problem* – rodzaj problemu: klasyfikacja (classification) lub regresja (regression)
* *set_name* – nazwa zbioru danych
* *number_of_samples* – liczba próbek w zbiorze; liczba naturalna

Uwagi: 
Zakładamy, że użytkownik podaje 'poprawne' wartości poszczególnych parametrów.
Przez liczby naturalne rozumiemy liczby całkowite dodatnie, tj. zbiór {1, 2, 3, …}.

Liczbę warstw i neuronów ukrytych w każdej warstwie można definiować na dwa sposoby.
Sposób 1: gdy parametr *layer_size* podamy jako pojedynczą liczbę, wówczas każda z *number_of_layers* warstw zawiera *layer_size* neuronów.
Sposób 2: gdy parametr *layer_size* podamy jako listę liczb oddzielonych przecinkami, wówczas i-ta warstwa zawiera *layer_size*[i] neuronów, a *number_of_layers* jest równe długości listy *layer_size*.

Pola *set_name* i *number_of_samples* stanowią części nazwy wczytywanego zbioru danych (nazwy plików są postaci *data.set_name.train.number_of_samples.csv* dla zbiorów listę treningowych oraz *data.set_name.test.number_of_samples.csv* dla zbiorów testowych).

# Wyniki przeprowadzonych eksperymentów:

Wczytanie plików png, zawierających wykresy przedstawiające wyniki działania sieci:

In [33]:
import os,glob

rootdir = '/home/kasia/Pulpit/Studia/MGU/projekt1/MG--Projekt1/Porownania'

classification_files = []
regression_files = []

# Klasyfikacja:
for subdir, dirs, files in os.walk(rootdir+"/Classification"):
    for directory in dirs:
        if directory != None and not os.path.exists(directory):
            os.mkdir(directory)
    for file in sorted(files):
        filename, ext = os.path.splitext(file)
        if ext != '.txt':
            classification_files.append('files/Porownania/Classification/'+file)
        
# Regresja
for subdir, dirs, files in os.walk(rootdir+"/Regression"):
    for directory in dirs:
        if directory != None and not os.path.exists(directory):
            os.mkdir(directory)
    for file in sorted(files):
        filename, _ = os.path.splitext(file)
        regression_files.append('files/Porownania/Regression/'+file)

1. Wyniki - Klasyfikacja

In [34]:
from IPython.display import HTML, display

for i in range(int(len(classification_files)/2)):
    print(classification_files[2*i][len('files/Porownania/Classification/'):])
    display(HTML("<table><tr><td><img src='"+classification_files[i]+"'></td><td><img src='"+classification_files[i+1]+"'></td></tr></table>"))

XOR100.png


,


XOR10000.png


,


XOR1000loss.png


...

,


three_gauss10000loss.png


,


three_gauss100loss.png


,


2. Wyniki - Regresja

In [39]:
from IPython.display import HTML, display

for i in range(int(len(regression_files)/2)):
    print(regression_files[2*i][len('files/Porownania/Regression/'):])
    display(HTML("<table><tr><td><img src='"+regression_files[i]+"'></td><td><img src='"+regression_files[i+1]+"'></td></tr></table>"))

activation100.png


,


activation10000.png


,


activation1000loss.png


...

,


square1000loss.png


,


square500.png


,
